In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import preprocessing
from sklearn import model_selection

from recsys import preprocess
from recsys import utility

RANDOM_STATE = 2342

%matplotlib inline

In [2]:
train = pd.read_csv('data/train_final.csv', delimiter='\t')
playlists = pd.read_csv('data/playlists_final.csv', delimiter='\t')
target_playlists = pd.read_csv('data/target_playlists.csv', delimiter='\t')
target_tracks = pd.read_csv('data/target_tracks.csv', delimiter = '\t')
tracks = pd.read_csv('data/tracks_final.csv', delimiter='\t')

In [3]:
x = pd.DataFrame(train['playlist_id'].drop_duplicates())
x.index = train['playlist_id'].unique()
x['tracks'] = train.groupby('playlist_id').apply(lambda x : x['track_id'].values)

In [28]:
x['size'] = x['tracks'].apply(lambda x : len(x))

x = x[x['size'] > 2]

In [31]:
np.setdiff1d(target_playlists['playlist_id'].values, x['playlist_id'].values)

array([], dtype=int64)

In [59]:
track_playlists = pd.DataFrame()
track_playlists['track_id'] = train['track_id'].unique()
track_playlists.index = train['track_id'].unique()

In [60]:
track_playlists['playlist_ids'] = train.groupby('track_id').apply(lambda x : x['playlist_id'].values)

In [61]:
track_playlists['size'] = track_playlists['playlist_ids'].apply(lambda x : len(x))

In [62]:
tr = track_playlists['track_id'].values

In [64]:
np.setdiff1d(target_tracks['track_id'].values, tr)

array([3626362])

In [67]:
sum(tracks['duration'] == -1)

2789

In [70]:
tr = tracks[tracks['duration'] == -1]

In [74]:
len(np.intersect1d(tr['track_id'].values, target_tracks['track_id'].values))

758

In [75]:
758/10000

0.0758

In [79]:
len(tracks['album'].unique())

27606

In [82]:
len(tracks['artist_id'].unique())

17536

In [ ]:
%%time

from recsys.preprocess import *
from sklearn import model_selection
import numpy as np
from recsys.utility import *

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

train = get_train()
target_playlist = get_target_playlists()
target_tracks = get_target_tracks()


train, test, target_playlist, target_tracks = train_test_split(train, test_size=0.20)

most_popular = get_most_popular_tracks(train)
tracks_in_playlist = get_playlist_track_list2(train)

tracks_to_suggest = most_popular.index.values
predictions = []

predictions = pd.DataFrame(target_playlist)
predictions.index = target_playlist['playlist_id']
predictions['track_ids'] = [np.array([]) for i in range(len(predictions))]
predictions['track_ids'] = predictions['track_ids'].astype(object)

for it,row in target_playlist.iterrows():
    count = 0
    i = 0
    pred = []
    while count < 5:
        if tracks_to_suggest[i] not in tracks_in_playlist.loc[row['playlist_id']]['track_ids']:
            # Predict track i
            # IMPORTANT: should we check if the track to suggest is in target_tracks?
            pred.append(tracks_to_suggest[i])
            count += 1
        i += 1
    #predictions.loc[row['playlist_id'],'track_ids'] = np.array([])
    predictions.loc[row['playlist_id']] = predictions.loc[row['playlist_id']].set_value('track_ids', np.array(pred))
    #predictions.append(' '.join(map(str, pred)))

#results = pd.DataFrame(np.transpose([target_playlist['playlist_id'].values, predictions]), columns=['playlist_id', 'track_ids'])

#results.to_csv('results.csv', index=False)


In [ ]:
def evaluate(test, recommendations, should_transform_test=True):
    """
     - "test" is:
           if should_transform_test == False: a dataframe with columns "playlist_id" and "track_id".
           else: a dict with "playlist_id" as key and a list of "track_id" as value.
     - "recommendations" is a dataframe with "playlist_id" and "track_id" as numpy.ndarray value.
    """
    if should_transform_test:
        # Tranform "test" in a dict:
        #   key: playlist_id
        #   value: list of track_ids
        test_df = preprocess.get_playlist_track_list2(test)
    else:
        test_df = test
    
    print(test_df.iloc[:10])
    
    mean_ap = 0
    for _,row in recommendations.iterrows():
        pl_id = row['playlist_id']
        tracks = row['track_ids']
        correct = 0
        ap = 0
        for it, t in enumerate(tracks):
            if t in test_df.loc[pl_id]['track_ids']:
                correct += 1
                ap += correct / (it+1)
        ap /= len(tracks)
        mean_ap += ap

    return mean_ap / len(recommendations)

In [ ]:
test.head()

In [ ]:
evaluate(recommendations=predictions, test=test)

In [ ]:
predictions['track_ids'] = predictions['track_ids'].apply(lambda x : ' '.join(map(str, x)))

In [ ]:
predictions

In [ ]:
from recsys.utility import *

In [ ]:
evaluate(recommendations=predictions, test=test)

In [ ]:
a = predictions.loc[10024884]

In [ ]:
a.set_value('track_ids', np.array([1, 23]))

In [ ]:
predictions.loc[10024884] = predictions.loc[10024884].set_value('track_ids', np.array([1, 2, 3]))

In [ ]:
predictions

In [ ]:
predictions.loc[10024884].set_value

In [ ]:
a = df['tracks']

In [ ]:
a.dtype = np.array

In [ ]:
train, test = model_selection.train_test_split(train, test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
small_train = train.loc[1:1000,:]

In [ ]:
s = small_train.groupby('track_id')

In [ ]:
a = s.count()

In [ ]:
a.sort_values('playlist_id', ascending=False, inplace=True)

In [ ]:
df.to_csv?

In [ ]:
a = pd.DataFrame(columns=['a', 'b'])

In [ ]:
train = get_train()
target_playlist = get_target_playlists()
target_tracks = get_target_tracks()

train, test = model_selection.train_test_split(train, test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
most_popular = get_most_popular_tracks(train)
tracks_in_playlist = get_playlist_track_list(train)

In [ ]:
most_popular = most_popular.index.values

In [ ]:
from recsys.preprocess import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import preprocessing
from sklearn import model_selection

RANDOM_STATE = 42

tracks_to_suggest = np.intersect1d(most_popular, target_tracks)
predictions = []

for it,row in target_playlist.iterrows():
    count = 0
    i = 0
    pred = []
    while count < 5:
        if tracks_to_suggest[i] not in tracks_in_playlist[row['playlist_id']]:
            # Predict track i
            pred.append(most_popular[i])
            count += 1
        i += 1
    predictions.append(' '.join(map(str, pred)))

In [ ]:
results = pd.DataFrame(np.transpose([target_playlist['playlist_id'].values, predictions]), columns=['playlist_id', 'track_ids'])

In [ ]:
len(test)

In [ ]:
from recsys.utility import *

In [ ]:
def evaluate(test, recommendations):
    test_good = preprocess.get_playlist_track_list(test)

    mean_ap = 0
    for pl_id, tracks in recommendations:
        correct = 0
        ap = 0
        for it, t in enumerate(tracks):
            if t in test_good[pl_id]:
                correct += 1
                ap += correct / (it+1)
        ap /= len(tracks)
        mean_ap += ap

    return mean_ap / len(recommendations)


In [ ]:
recomm = np.array([list(map(int,x.split(' '))) for x in results['track_ids'].values])

In [ ]:
recomm

In [ ]:
evaluate(test, recomm)

In [ ]:
def train_test_split(train, test_size=0.3):
    playlists = train.groupby('playlist_id').count()
    to_choose_playlists = playlists[playlists['track_id'] > 7].index.values
    target_playlists = np.random.choice(to_choose_playlists, replace=False, size=int(test_size * len(to_choose_playlists)))
    
    target_tracks = np.array([])
    indexes = np.array([])
    for p in target_playlists:
        selected_df = train[train['playlist_id'] == p].sample(5)
        selected_tracks = selected_df['track_id'].values
        target_tracks = np.union1d(target_tracks, selected_tracks)
        indexes = np.union1d(indexes, selected_df.index.values)
    
    test = df.loc[indexes].copy()
    train = train.drop(indexes)
    
    return train, test, target_playlists, target_tracks
    

In [ ]:
x,y,z,w = train_test_split(train)

In [ ]:
len(train)

In [ ]:
len(x)

In [ ]:
len(y)

In [ ]:
len(z)

In [ ]:
len(w)

In [ ]:
len(y)/len(x)

In [ ]:
df.to_csv?

In [ ]:
train[train['track_id'] == 252]

In [ ]:
most_popular = get_most_popular_tracks(train)

In [ ]:
most_popular

In [ ]:
target_tracks['track_id'].values

In [ ]:
len(np.intersect1d(most_popular.index.values[:10], target_tracks['track_id'].values))

In [ ]:
len(target_tracks['track_id'])

In [ ]:
len(most_popular.index)

In [ ]:
most_popular

In [ ]:
playlists = train.groupby('playlist_id').count()
to_choose_playlists = playlists[playlists['track_id'] > 7].index.values

In [ ]:
len(to_choose_playlists)

In [ ]:
len(playlists)

In [ ]:
playlists = a.groupby('playlist_id').count()

In [ ]:
df.loc[[518209,598021]]

In [ ]:
a = np.array([1, 2, 3, 4, 5])

In [ ]:
np.union1d(a,b)

In [ ]:
a = df.loc[[518209, 598021]]

In [ ]:
len(df.drop([518209]))

In [ ]:
len(df)

In [ ]:
train[train['playlist_id'] == 6226465].sample(5)['track_id'].values

In [ ]:
len(to_choose_playlists)

In [ ]:
a = (train.groupby('playlist_id').count())

In [ ]:
a[a['track_id'] > 10].index.values

In [ ]:
pd.Series?

In [ ]:
a = pd.Series(dtype=np.ndarray)

In [ ]:
a

In [ ]:
len(target_tracks)

In [ ]:
len(most_popular)

In [ ]:
len(np.intersect1d(target_tracks, most_popular))